In [24]:
import requests
import csv
from bs4 import BeautifulSoup
import re
teacherListPath = './data/teacherList.csv'
baseTeacherUrl = 'http://jszy.hhu.edu.cn'

### 获取教师列表

In [28]:
def TchList():
    """index range : 1 - 110"""

    myUrl = 'https://jszy.hhu.edu.cn/_wp3services/generalQuery?queryObj=teacherHome'

    tchHeaders = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',

        'Content-Length': '964',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 's1_1=Mac=253cd8dc-2991-4596-a98e-f04ee8a6a9bc&OpenID=&LastAloneTime=2022/4/22 20:29:40&LastVisitTime=2022/4/22 20:30:27; iPlanetDirectoryPro=AQIC5wM2LY4Sfcy1qxXnERvdhoxyR9NTibA5+VkfgVXlSY8=@AAJTSQACMDI=#; JSESSIONID=9EE6AA686BA419F839C6F471CD8CCE03',
        'Host': 'jszy.hhu.edu.cn',
        'Origin': 'https://jszy.hhu.edu.cn',
        'Referer': 'https://jszy.hhu.edu.cn/jsflcx/list.htm',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Microsoft Edge";v="101"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',

        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',

        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',
        'X-Requested-With': 'XMLHttpRequest'
    }

    tchData = {
        'pageIndex': 1,
        'rows': 1000,
        'conditions': '[{"field": "language", "value": "1", "judge": "="}, {"field": "published", "value": "1", "judge": "="}]',
        'returnInfos': '[{"field": "cnUrl", "name": "cnUrl"},\
                        {"field": "title", "name": "title"},\
                        {"field": "department", "name": "department"},\
                        {"field": "career", "name": "career"},\
                        {"field": "exField1", "name": "exField1"}\
                        ]',
        'articleType': 1,
        'level': 0,
        'pageEvent': 'doSearchByPage'
    }

    resp = requests.post(url=myUrl, data=tchData, headers=tchHeaders)

    return [{'教师名': datai['title'],'职称':datai['career'], '招生资格':datai['exField1'],
             '部门':datai['department'], '主页链接':datai['cnUrl']}
            for datai in resp.json()['data']]

In [29]:
teacherList = TchList()
# teacherDataList[2]

### 存取数据为csv格式

In [30]:
with open(teacherListPath,'w') as file:
    teacherListHeader = ['教师名','职称','招生资格','部门','主页链接']
    csvWriter = csv.DictWriter(file,teacherListHeader,delimiter='\t',lineterminator='\n')
    csvWriter.writeheader()
    csvWriter.writerows(teacherDataList)

In [4]:
with open(teacherListPath,'r') as file:
    csvReader = csv.reader(file,delimiter='\t')
    teacherList = [row for row in csvReader]
teacherList[2]

['安天庆', '教授', '博士生导师', '理学院', 'http://jszy.hhu.edu.cn/atq']

### 爬取每个教师的数据 测试

In [4]:
with open(teacherListPath,'r') as file:
    csvReader = csv.reader(file,delimiter=',')
    teacherList = [row for row in csvReader]
teacherList[2]
teacherLink = teacherList[128][-1]
teacherLink = 'http://jszy.hhu.edu.cn/bx102'

In [5]:
headersi = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',}

def GetSoup(url):
    response = requests.get(url,headers=headersi)
    response.encoding='utf-8'
    return BeautifulSoup(response.text, 'html.parser')
pageiSoup = GetSoup(teacherLink)

In [13]:
def GetAbsText(text):
    return re.sub('\r|\t|\n|\xa0| ','',text)

def GetAbsTextWithSpace(text):
    return re.sub('\r|\t|\n|\xa0','',text)

In [ ]:
infoDict = {}
infoDict['头像']=baseTeacherUrl+pageiSoup.find(name='img',attrs={"alt":'头像'}).get('src')
for infoi in pageiSoup.find_all(name='p',attrs={"class":'news_text'}): 
  keyi = GetAbsText(infoi.find(name='span',attrs={'class':'title'}).text).replace('：','')
  valuei = ' '.join([valuei.text.strip() for valuei in infoi.find_all(name='span',attrs={'class':'txt'})])
  infoDict[keyi] = valuei
infoDict

In [ ]:
iframeLink = baseTeacherUrl + pageiSoup.find(name='iframe').get('src')
relatedTchsSoup = GetSoup(iframeLink).find_all(name='div',attrs={'class':'news_title'})
infoDict['相关教师'] = [GetAbsText(tchSoup.text) for tchSoup in relatedTchsSoup]
infoDict['相关教师']

In [16]:
allTitleSoup = pageiSoup.find_all(name='div',attrs={'class':'tt'})

In [ ]:
# 个人简介
personalBaseSoup = allTitleSoup[0].findNextSibling()
infoDict['个人简介'] = GetAbsTextWithSpace(personalBaseSoup.text)
infoDict['个人简介']

In [ ]:
# 个人资料
personalDataSoup = allTitleSoup[1].findNextSibling()
for infoi in personalDataSoup.find_all(name='li'):
    keyi = GetAbsText(infoi.find(name='span',attrs={'class':'title'}).text).replace('：','')
    valuei = infoi.find(name='span',attrs={'class':'txt'}).text
    infoDict[keyi] = valuei
infoDict

In [ ]:
# 教育经历,工作经历,研究领域,科研项目,论文,科技成果,开授课程,教学成果,教学资源,社会职务,荣誉及奖励
for titleSoup in allTitleSoup[2:-1]:
    titleText = GetAbsText(titleSoup.text)
    contentSoup = titleSoup.findNextSibling()
    allContentSoup = contentSoup.find_all(name='li', attrs={'class': 'news'})
    if len(allContentSoup) != 0:
        infoDict[titleText] = [GetAbsTextWithSpace(coutenti.text)
                               for coutenti in allContentSoup]
    else:
        allContentSoup = contentSoup.find_all(name='p', attrs={'class': 'MsoNormal'})
        if len(allContentSoup) != 0:
            infoDict[titleText] = [GetAbsTextWithSpace(
                coutenti.text) for coutenti in allContentSoup]
        else:
            infoDict[titleText] = GetAbsTextWithSpace(contentSoup.text)
infoDict


### 爬取每个老师的数据

In [25]:
def GetSoup(url):
    headersi = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',}
    response = requests.get(url,headers=headersi)
    response.encoding='utf-8'
    return BeautifulSoup(response.text, 'html.parser')

def GetAbsText(text):
    return re.sub('\r|\t|\n|\xa0| ','',text)

def GetAbsTextWithSpace(text):
    return re.sub('\r|\t|\n|\xa0','',text)

In [26]:
def GetTeacherInfo(teacherName,teacherLink):
    infoDict = {'主页链接':teacherLink}
    teacherSoup = GetSoup(teacherLink)
    
    notVisited = ['404 Not Found', '403 Forbidden']
    if teacherSoup.title.text in notVisited:
        infoDict['姓名'] = teacherName
        return infoDict

    # 左边栏介绍 头像链接 职称 部门 学科 教学部门 电话 邮件 办公地址 通讯地址 邮编
    infoDict['头像链接']=baseTeacherUrl+teacherSoup.find(name='img',attrs={"alt":'头像'}).get('src')
    
    for infoi in teacherSoup.find_all(name='p',attrs={"class":'news_text'}): 
        keyi = GetAbsText(infoi.find(name='span',attrs={'class':'title'}).text).replace('：','')
        valuei = ' '.join([valuei.text.strip() for valuei in infoi.find_all(name='span',attrs={'class':'txt'})])
        infoDict[keyi] = valuei
    
    # 相关教师
    iframeLink = baseTeacherUrl + teacherSoup.find(name='iframe').get('src')
    relatedTchsSoup = GetSoup(iframeLink).find_all(name='div',attrs={'class':'news_title'})
    infoDict['相关教师'] = ';'.join([GetAbsText(tchSoup.text) for tchSoup in relatedTchsSoup])

    allTitleSoup = teacherSoup.find_all(name='div',attrs={'class':'tt'})

    # 个人简介
    personalBaseSoup = allTitleSoup[0].findNextSibling()
    infoDict['个人简介'] = GetAbsTextWithSpace(personalBaseSoup.text)
    
    # 个人资料: 姓名 出生年月 性别 学历 学位 毕业院校
    personalDataSoup = allTitleSoup[1].findNextSibling()
    for infoi in personalDataSoup.find_all(name='li'):
        keyi = GetAbsText(infoi.find(name='span',attrs={'class':'title'}).text).replace('：','')
        valuei = infoi.find(name='span',attrs={'class':'txt'}).text
        infoDict[keyi] = valuei
    
    # 教育经历,工作经历,研究领域,科研项目,论文,科技成果,开授课程,教学成果,教学资源,社会职务,荣誉及奖励,招生信息
    for titleSoup in allTitleSoup[2:]:
        titleText = GetAbsText(titleSoup.text)
        contentSoup = titleSoup.findNextSibling()
        allContentSoup = contentSoup.find_all(name='li', attrs={'class': 'news'})
        if len(allContentSoup) != 0:
            infoDict[titleText] = '\n'.join([GetAbsTextWithSpace(coutenti.text)
                                for coutenti in allContentSoup])
        else:
            allContentSoup = contentSoup.find_all(name='p')
            if len(allContentSoup) != 0:
                infoDict[titleText] = '\n'.join([GetAbsTextWithSpace(
                    coutenti.text) for coutenti in allContentSoup])
            else:
                infoDict[titleText] = GetAbsTextWithSpace(contentSoup.text)
    
    return infoDict

teacherName,teacherLink = '陈达','https://jszy.hhu.edu.cn/cd/'

teacherName,teacherLink = '冯钧','https://jszy.hhu.edu.cn/fj12/'

# GetTeacherInfo(teacherName,teacherLink)

In [27]:
with open(teacherListPath, 'r') as file:
    csvReader = csv.reader(file, delimiter='\t')
    teacherLinkList = [(row[0], row[-1]) for row in csvReader]

In [37]:
teacherInfoList = []

In [ ]:
for tchName, tchLink in teacherLinkList[329:]:
    print(tchName, tchLink)
    # break
    teacherInfoList.append(GetTeacherInfo(tchName, tchLink))

In [40]:
len(teacherInfoList)

647

In [41]:

with open('./data/teacherInfoWithSpace.csv', 'a', encoding='utf-8') as file:
    teacherInfoListHeader = ['姓名', '出生年月', '性别', '学历', '学位', '毕业院校', '主页链接',
                             '头像链接', '职称', '部门', '学科', '教学部门', '电话', '邮件', '办公地址', '通讯地址', '邮编', '相关教师',
                             '个人简介', '教育经历', '工作经历', '研究领域', '科研项目', '论文', '科技成果',
                             '开授课程', '教学成果', '教学资源', '社会职务', '荣誉及奖励', '招生信息']
    csvWriter = csv.DictWriter(file, teacherInfoListHeader, delimiter='\t', lineterminator='\n')
    # csvWriter.writeheader()
    csvWriter.writerows(teacherInfoList)


### 添加教师编号

In [42]:
import pandas as pd
teacherInfoPath = './data/teacherInfoWithSpace.csv'
tchInfos = pd.read_csv(teacherInfoPath,header=0,delimiter='\t')
tchInfos.columns

Index(['姓名', '出生年月', '性别', '学历', '学位', '毕业院校', '主页链接', '头像链接', '职称', '部门',
       '学科', '教学部门', '电话', '邮件', '办公地址', '通讯地址', '邮编', '相关教师', '个人简介', '教育经历',
       '工作经历', '研究领域', '科研项目', '论文', '科技成果', '开授课程', '教学成果', '教学资源', '社会职务',
       '荣誉及奖励', '招生信息'],
      dtype='object')

In [43]:
indexs =tchInfos.index.to_series()
indexs.name = '教师编号'
tchInfosWithID =  pd.concat([indexs,tchInfos],axis=1)
tchInfos.to_csv('./data/teacherInfoWithSpaceWithID.csv',sep='\t',index_label='教师编号')
# tchInfos.to_csv('./data/teacherInfoWithID1.csv',sep='\t',index=0)

### 读取老师信息

In [44]:
import pandas as pd
pd.set_option('display.max_columns',10)
pd.set_option('display.max_colwidth',10)
teacherInfoPath = './data/teacherInfoWithSpaceWithID.csv'
tchInfos = pd.read_csv(teacherInfoPath,header=0,delimiter='\t',index_col='教师编号')
tchInfos.head()

,姓名,出生年月,性别,学历,学位,...,教学成果,教学资源,社会职务,荣誉及奖励,招生信息
教师编号,,,,,,,,,,,
0,安天庆,1963-0...,男,博士研究生,博士学位,...,NaN,NaN,江苏省数学会...,NaN,每年招收博士...
1,安杰晶,1988-1...,女,硕士研究生,硕士学位,...,NaN,NaN,NaN,NaN,NaN
2,卞培良,1993-0...,男,博士研究生,工学博士学位,...,NaN,NaN,NaN,NaN,NaN
3,边钰,1995-1...,女,硕士研究生,硕士学位,...,NaN,NaN,NaN,NaN,NaN
4,毕利东,1980-1...,男,博士研究生,博士学位,...,NaN,NaN,NaN,NaN,NaN
